
## MaxwellFDFD Data download
1. download data from link
2. make directory
3. unzip dataset


In [7]:
!curl -L -o maxwellfdfd.zip https://drive.google.com/uc?id=14-Bl89OzRtLM1MCW2H81Xvivq8EvTrmB

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:04 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:05 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:06 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:07 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:08 --:--:--     0
  0     0    0     0    0     0      0      0 --:--

In [10]:
#!mkdir maxwellfdfd
#!unzip maxwellfdfd.zip -d ./maxwellfdfd

## ML(Random Forest regressor):
1. create model for Multi-output Random Forest regressor

In [11]:
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
%matplotlib inline

In [12]:
import os
data_dir = os.path.join(os.getcwd(), 'maxwellfdfd')
data_dir

'D:\\2.CEM\\maxwellfdfd-controlgan\\maxwellfdfd'

In [15]:
def compress_image(prev_image, n):
    if n < 2:
        return prev_image

    height = prev_image.shape[0] // n
    width = prev_image.shape[1] // n
    new_image = np.zeros((height, width), dtype=int)
    for i in range(0, height):
        for j in range(0, width):
            new_image[i, j] = prev_image[n * i, n * j]

    return new_image

DATASETS_TRAIN = [
    'binary_501',
    'binary_502',
    'binary_503',
    'binary_504',
    'binary_505',
    'binary_506',
    'binary_507',
    'binary_508',
    'binary_509',
    'binary_510',
    'binary_511',
    'binary_512',
    'binary_1001',
    'binary_1002',
    'binary_1003',
    'binary_rl_fix_501',
    'binary_rl_fix_502',
    'binary_rl_fix_503',
    'binary_rl_fix_504',
    'binary_rl_fix_505',
    'binary_rl_fix_506',
    'binary_rl_fix_507',
    'binary_rl_fix_508',
    'binary_rl_fix_509',
    'binary_rl_fix_510',
    'binary_rl_fix_511',
    'binary_rl_fix_512',
    'binary_rl_fix_513',
    'binary_rl_fix_514',
    'binary_rl_fix_515',
    'binary_rl_fix_516',
    'binary_rl_fix_517',
    'binary_rl_fix_518',
    'binary_rl_fix_519',
    'binary_rl_fix_520',
    'binary_rl_fix_1001',
    'binary_rl_fix_1002',
    'binary_rl_fix_1003',
    'binary_rl_fix_1004',
    'binary_rl_fix_1005',
    'binary_rl_fix_1006',
    'binary_rl_fix_1007',
    'binary_rl_fix_1008',
]

DATASETS_TEST = [
    'binary_new_test_501',
    'binary_new_test_1501',
    'binary_test_1101',
]

def cem_dataset(train=True):
    if train:
        DATAPATH = os.path.join(data_dir, 'train')
        DATASETS = DATASETS_TRAIN
    else:
        DATAPATH = os.path.join(data_dir, 'test')
        DATASETS = DATASETS_TEST
    width = 40
    height = 20
    input_data = []
    targets = []
    # load Train dataset
    for data in DATASETS:
        dataframe = pd.read_csv(os.path.join(DATAPATH, '{}.csv'.format(data)), delim_whitespace=False, header=None)
        dataset = dataframe.values

        # split into input (X) and output (Y) variables
        fileNames = dataset[:, 0]

        # 1. first try max
        dataset[:, 1:25] /= 2767.1

        # 2. Classification or Regression,
        labels = dataset[:, 1:25]
        labels = np.apply_along_axis(lambda x: np.argmax(x), 1, labels)
        targets.extend(labels)

        for idx, file in enumerate(fileNames):
            try:
                image = Image.open(os.path.join(DATAPATH, data, '{}.tiff'.format(int(file))))
                image = np.array(image).astype(np.uint8)
            except (TypeError, FileNotFoundError) as te:
                image = Image.open(os.path.join(DATAPATH, data, '{}.tiff'.format(idx + 1)))
                try:
                    image = np.array(image).astype(np.uint8)
                except:
                    continue
            image = compress_image(image, 5)
            input_data.append(np.array(image).flatten(order='C'))
    input_data = np.vstack(input_data).reshape(-1, 1, height, width)
    input_data = input_data.transpose((0, 1, 2, 3))  # convert to HWC CHW
    input_data = input_data.reshape(input_data.shape[0], -1)
    print(input_data.shape)
    print(f'Data Loading Finished. len : {len(input_data)}')
    return pd.DataFrame(targets), input_data


In [ ]:
X_train, y_train = cem_dataset(train=True)

In [ ]:
X_train = X_train - 12

In [ ]:
good_idx = X_train >= 0
X_train = X_train[good_idx].dropna()
X_train

In [ ]:
good_idx = good_idx.to_numpy().reshape(-1)
y_train = y_train[good_idx, :]

In [ ]:
X_test, y_test = cem_dataset(train=False)

In [ ]:
model = MultiOutputClassifier(RandomForestClassifier(n_estimators=100))

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
y_pred_df=pd.DataFrame(y_pred)

In [ ]:
X_test

In [ ]:
y_pred_image = y_pred.reshape(y_pred.shape[0], 1, 20, 40)

In [ ]:
import torch
from torchvision.utils import make_grid

def denorm(img_tensors):
    return img_tensors * 1.

def show_images(images, nmax=64):
    fig, ax = plt.subplots(figsize=(8, 8))
    ax.set_xticks([]); ax.set_yticks([])
    s = make_grid(images.detach()[:nmax], nrow=8)
    ax.imshow(make_grid(denorm(images.detach()[:nmax]), nrow=8, padding=5, pad_value=0.5).permute(1, 2, 0))

In [ ]:
show_images(torch.from_numpy(y_pred_image))

In [ ]:
X_test[:10]

In [ ]:
def get_prediction_model():
    MODEL_JSON_PATH = 'models/cnn_small_rmse_128_300/rmse_rect_1.json'
    MODEL_H5_PATH = 'models/cnn_small_rmse_128_300/rmse_rect_1.h5'

    # load json and create model
    json_file = open(MODEL_JSON_PATH, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)

    # load weights into new model
    loaded_model.load_weights(MODEL_H5_PATH)
    return loaded_model

In [ ]:
prediction_model = get_prediction_model()

In [ ]:
n_classes = 12
img_height = 20
img_width = 40
match_cnt = 0
correct_cnt_top3 = 0
correct_cnt_top5 = 0
result_match = dict()
result_top3 = dict()
result_top5 = dict()
result_truth = dict()
# Initialize result dict
for n in range(n_classes):
    wavelength = n * 50 + 1000
    result_match[str(wavelength)] = 0
    result_top3[str(wavelength)] = 0
    result_top5[str(wavelength)] = 0
    result_truth[str(wavelength)] = 0
print('# of samples', y_pred.shape[0])

labels = []
predictions = []

for i in range(y_pred.shape[0]):
    class_int = (i % n_classes)
    
    predictions.append(class_int)
    
    wavelength = class_int * 50 + 1000
#     single_image = y_pred_image[i] // 255
    single_image_for_model = y_pred[i].reshape((1, img_height, img_width, 1))
    real = prediction_model.predict(single_image_for_model)

    argsort_top5 = (-real).argsort()[:, :5][0] - 12
    argsort_top3 = (-real).argsort()[:, :3][0] - 12
    
    argsort_top5[argsort_top5 < 0] = 0
    argsort_top3[argsort_top3 < 0] = 0
    label = argsort_top3[0]
    labels.append(label)
    if class_int in argsort_top5:
        result_top5[str(wavelength)] += 1

    if class_int in argsort_top3:
        result_top3[str(wavelength)] += 1

    if class_int == argsort_top3[0]:
        result_match[str(wavelength)] += 1

    if argsort_top3[0] > -1:
        result_truth[str(argsort_top3[0]* 50 + 1000)] += 1
        

print(f'match_cnt : {sum(result_match.values())}, \\t correct_cnt_top3 : {sum(result_top3.values())}, \\t correct_cnt_top5 : {sum(result_top5.values())}')

percent_match = sum(result_match.values())
percent_top3 = sum(result_top3.values())
percent_top5 = sum(result_top5.values())

print('percent_match : {0:.4f} \\t top3 : {1:.4f} \\t top5 : {2:.4f}'.format(percent_match, percent_top3, percent_top5))

In [ ]:
result = np.asarray(result)
result[result<0] = -1
predictions = result
predictions

In [ ]:
labels = X_test.reshape(-1)
labels

In [ ]:
from sklearn.metrics import precision_score, recall_score, classification_report, confusion_matrix

cm = confusion_matrix(labels, predictions)
print(predictions)
print(labels)

# Print the confusion matrix
print(cm)
# Print the precision and recall, among other metrics
print(classification_report(labels, predictions, digits=3))
recall = recall_score(labels, predictions, average='micro')
precision = precision_score(labels, predictions, average='micro')
fscore = 2 * recall * precision / (recall + precision)

print(f'recall : {recall:.3f}, precision: {precision:.3f}, f-score: {fscore:.3f}')
  

    